<font size=6> The total sovereign emissions calculated by adding up scope 1, 2 and 3 emissions  </font>

In [1]:
# 'capture' magic prevents long outputs from spamming your notebook
#%%capture pipoutput

# For loading predefined environment variables from files
# Typically used to load sensitive access credentials
%pip install python-dotenv

# Standard python package for interacting with S3 buckets
%pip install boto3
# Interacting with Trino and using Trino with sqlalchemy
%pip install trino sqlalchemy sqlalchemy-trino

# Pandas and parquet file i/o
%pip install pandas pyarrow fastparquet

# OS-Climate utilities to make data ingest easier
%pip install osc-ingest-tools
%pip install ipython --upgrade
%pip install country_converter --upgrade
%pip install pint-pandas
%pip install openscm-units
%pip install pint
%pip install jedi
%pip install pycountry

You should consider upgrading via the '/opt/app-root/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/opt/app-root/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/opt/app-root/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/opt/app-root/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/opt/app-root/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/opt/app-root/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the 

In [2]:
from dotenv import dotenv_values, load_dotenv
import osc_ingest_trino as osc
import os
import pathlib
dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

In [3]:
import trino
from sqlalchemy.engine import create_engine

env_var_prefix = 'TRINO'

sqlstring = 'trino://{user}@{host}:{port}/'.format(
    user = os.environ[f'{env_var_prefix}_USER'],
    host = os.environ[f'{env_var_prefix}_HOST'],
    port = os.environ[f'{env_var_prefix}_PORT']
)
sqlargs = {
    'auth': trino.auth.JWTAuthentication(os.environ[f'{env_var_prefix}_PASSWD']),
    'http_scheme': 'https',
    'catalog': 'osc_datacommons_dev'
}
engine = create_engine(sqlstring, connect_args = sqlargs)
connection = engine.connect()

trino_bucket = osc.attach_s3_bucket("S3_DEV")

/opt/app-root/lib64/python3.8/site-packages/trino/sqlalchemy/dialect.py:286: SAWarning: Dialect trino:rest will not make use of SQL compilation caching as it does not set the 'supports_statement_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Dialect maintainers should seek to set this attribute to True after appropriate development and testing for SQLAlchemy 1.4 caching support.   Alternatively, this attribute may be set to False which will disable this warning. (Background on this error at: https://sqlalche.me/e/14/cprf)
  res = connection.execute(sql.text(query))


In [4]:
import boto3

s3_source = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ['S3_LANDING_ENDPOINT'],
    aws_access_key_id=os.environ['S3_LANDING_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_LANDING_SECRET_KEY'],
)
source_bucket = s3_source.Bucket(os.environ['S3_LANDING_BUCKET'])

In [5]:
# define source and destination tables
# LULUCF (Land Use, Land-Use Change and Forestry)
ingest_catalog = 'osc_datacommons_dev'
ingest_schema = 'pcaf_sovereign_footprint'
ingest_table = 'sf_total_sovereign_emissions'
src_table_1 = 'sf_oecd_imgr_fco2'
src_table_2 = 'sf_unfccc_results'


In [6]:
# Get the values saved in Trino related with Foreign CO2 emissions embodied in gross imports

import pandas as pd

sql=f"""
select country_iso_code,partner_iso_code,industry_code,validity_date,attribute,value,value_units from {ingest_catalog}.{ingest_schema}.{src_table_1} """ 
df1 = pd.read_sql(sql, engine)
df1
#df1["units"] = "kt"
#df1 = requantify_df(df1).convert_dtypes()
#df1.info(verbose=True)
#df1


,country_iso_code,partner_iso_code,industry_code,validity_date,attribute,value,value_units
0,ITA,PER,D35,2003,Foreign CO2 emissions embodied in gross imports,0.0,Mt CO2
1,ITA,PER,D35,2004,Foreign CO2 emissions embodied in gross imports,0.0,Mt CO2
2,ITA,PER,D35,2005,Foreign CO2 emissions embodied in gross imports,0.0,Mt CO2
3,ITA,PER,D35,2006,Foreign CO2 emissions embodied in gross imports,0.0,Mt CO2
4,ITA,PER,D35,2007,Foreign CO2 emissions embodied in gross imports,0.0,Mt CO2
...,...,...,...,...,...,...,...
334651,IDN,PER,D35,2006,Foreign CO2 emissions embodied in gross imports,0.0,Mt CO2
334652,IDN,PER,D35,2007,Foreign CO2 emissions embodied in gross imports,0.0,Mt CO2
334653,IDN,PER,D35,2008,Foreign CO2 emissions embodied in gross imports,0.0,Mt CO2
334654,IDN,PER,D35,2009,Foreign CO2 emissions embodied in gross imports,0.0,Mt CO2


<font size=3> Remove invalid country codes </font>

In [7]:
import pycountry
import pandas as pd
df_country = pd.DataFrame([country.__dict__['_fields'] for country in pycountry.countries])
df_country = df_country.rename(columns={'alpha_3':'country_iso_code'})
df_country=df_country[['country_iso_code']]
# add Rest of World to the country dataframe
dict_row = {'country_iso_code':'ROW'}
df_country = df_country.append(dict_row, ignore_index = True)
df_country.info(verbose=True)
df1=df1.convert_dtypes()
#df1.info(verbose=True)
df1 = pd.merge(df1,df_country,on=['country_iso_code'])  
df1=df1.convert_dtypes()
#df_result.drop(columns=['country_name_y'],inplace=True)
df1


/tmp/ipykernel_11791/2921960101.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_country = df_country.append(dict_row, ignore_index = True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 1 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   country_iso_code  250 non-null    object
dtypes: object(1)
memory usage: 2.1+ KB


,country_iso_code,partner_iso_code,industry_code,validity_date,attribute,value,value_units
0,ITA,PER,D35,2003,Foreign CO2 emissions embodied in gross imports,0.0,Mt CO2
1,ITA,PER,D35,2004,Foreign CO2 emissions embodied in gross imports,0.0,Mt CO2
2,ITA,PER,D35,2005,Foreign CO2 emissions embodied in gross imports,0.0,Mt CO2
3,ITA,PER,D35,2006,Foreign CO2 emissions embodied in gross imports,0.0,Mt CO2
4,ITA,PER,D35,2007,Foreign CO2 emissions embodied in gross imports,0.0,Mt CO2
...,...,...,...,...,...,...,...
270139,HKG,MMR,D35,2014,Foreign CO2 emissions embodied in gross imports,0.0,Mt CO2
270140,HKG,MMR,D35,2015,Foreign CO2 emissions embodied in gross imports,0.0,Mt CO2
270141,HKG,MMR,D35,2016,Foreign CO2 emissions embodied in gross imports,0.0,Mt CO2
270142,HKG,MMR,D35,2017,Foreign CO2 emissions embodied in gross imports,0.0,Mt CO2


Calcuate Scope 2:
GHG emissions occurring as a consequence of the domestic use of grid-supplied electricity, heat,
steam and/or cooling which is imported from another territory


In [8]:
rename_columns = {'value':'total','partner_iso_code':'country_iso_code','value_units' :'total_units'}
agg_columns = { 'value' : 'sum'}
columns_order = ['attribute','industry_code','country_iso_code','validity_date','total','total_units']
df1 = df1.groupby(['partner_iso_code','industry_code','attribute','validity_date','value_units'],as_index=False).agg(agg_columns).rename(columns=rename_columns).reindex(columns=columns_order)

df1.convert_dtypes()

df1.info(verbose=True)
df1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4032 entries, 0 to 4031
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   attribute         4032 non-null   string 
 1   industry_code     4032 non-null   string 
 2   country_iso_code  4032 non-null   string 
 3   validity_date     4032 non-null   Int64  
 4   total             4032 non-null   Float64
 5   total_units       4032 non-null   string 
dtypes: Float64(1), Int64(1), string(4)
memory usage: 197.0 KB


,attribute,industry_code,country_iso_code,validity_date,total,total_units
0,Foreign CO2 emissions embodied in gross imports,D35,APEC,1995,39.478,Mt CO2
1,Foreign CO2 emissions embodied in gross imports,D35,APEC,1996,39.637,Mt CO2
2,Foreign CO2 emissions embodied in gross imports,D35,APEC,1997,46.108,Mt CO2
3,Foreign CO2 emissions embodied in gross imports,D35,APEC,1998,47.416,Mt CO2
4,Foreign CO2 emissions embodied in gross imports,D35,APEC,1999,47.128,Mt CO2
...,...,...,...,...,...,...
4027,Foreign CO2 emissions embodied in gross imports,DTOTAL,ZSCA,2014,209.262,Mt CO2
4028,Foreign CO2 emissions embodied in gross imports,DTOTAL,ZSCA,2015,214.719,Mt CO2
4029,Foreign CO2 emissions embodied in gross imports,DTOTAL,ZSCA,2016,206.428,Mt CO2
4030,Foreign CO2 emissions embodied in gross imports,DTOTAL,ZSCA,2017,209.894,Mt CO2


In [9]:
# divide the dataframe into two dataframes 
# to calculate Scope2 and Scope3 Values
df_D35 = df1[df1['industry_code']=='D35']
df_D35 = df_D35[['country_iso_code','validity_date','total','total_units']]
df_DTOTAL = df1[df1['industry_code']=='DTOTAL']
df_DTOTAL = df_DTOTAL[['country_iso_code','validity_date','total','total_units']]
df_result = pd.merge(df_D35,df_DTOTAL,on=['country_iso_code','validity_date'])  
df_result=df_result.convert_dtypes()
#df_result.drop(columns=['attribute_y','total_units_y'],inplace=True)
rename_columns = {'total_x':'scope2_value','total_y':'scope3_value','total_units_x':'scope2_value_units','total_units_y':'scope3_value_units'}
df_result.rename(columns=rename_columns,inplace=True)
df_result['scope3_value'] = df_result['scope3_value'] - df_result['scope2_value'] 
df_result[df_result['country_iso_code']=='CAN']

,country_iso_code,validity_date,scope2_value,scope2_value_units,scope3_value,scope3_value_units
216,CAN,1995,5.129,Mt CO2,179.111,Mt CO2
217,CAN,1996,5.269,Mt CO2,188.198,Mt CO2
218,CAN,1997,6.362,Mt CO2,202.972,Mt CO2
219,CAN,1998,7.993,Mt CO2,214.108,Mt CO2
220,CAN,1999,9.1,Mt CO2,222.868,Mt CO2
221,CAN,2000,18.191,Mt CO2,236.036,Mt CO2
222,CAN,2001,19.531,Mt CO2,228.887,Mt CO2
223,CAN,2002,8.195,Mt CO2,233.5,Mt CO2
224,CAN,2003,8.22,Mt CO2,232.392,Mt CO2
225,CAN,2004,8.411,Mt CO2,238.375,Mt CO2


In [10]:
# Get the values saved in Trino containing  GHG emissions provided by UNFCCC & GDP PPP values from Worldbank

import pandas as pd

sql=f"""
select country_iso_code,validity_date,ghg_total_without_lulucf,ghg_total_without_lulucf_units,gdp,gdp_units
from {ingest_catalog}.{ingest_schema}.{src_table_2} """ 
df_unfccc = pd.read_sql(sql, engine)
df_unfccc
#df1["units"] = "kt"
#df1 = requantify_df(df1).convert_dtypes()
#df1.info(verbose=True)
#df1


,country_iso_code,validity_date,ghg_total_without_lulucf,ghg_total_without_lulucf_units,gdp,gdp_units
0,JPN,1990,1.269015e+06,CO2e * kt,2.466213e+12,USD
1,JPN,1991,1.283387e+06,CO2e * kt,2.636747e+12,USD
2,JPN,1992,1.295604e+06,CO2e * kt,2.719707e+12,USD
3,JPN,1993,1.291285e+06,CO2e * kt,2.769754e+12,USD
4,JPN,1994,1.352191e+06,CO2e * kt,2.856993e+12,USD
...,...,...,...,...,...,...
1144,CYP,2013,7.918777e+03,CO2e * kt,2.618148e+10,USD
1145,CYP,1993,6.798931e+03,CO2e * kt,9.293831e+09,USD
1146,CYP,1994,7.049134e+03,CO2e * kt,1.005234e+10,USD
1147,CYP,1995,6.985579e+03,CO2e * kt,1.112138e+10,USD


In [11]:
def requantify_df(df):
    units_col = None
    columns_reversed = reversed(df.columns)
    for col in columns_reversed:
        if col.endswith("_units"):
            if units_col:
                # We expect _units column to follow a non-units column
                raise ValueError
            units_col = col
            continue
        if units_col:
            if col + '_units' != units_col:
                raise ValueError
            if (df[units_col]==df[units_col][0]).all():
                # Make a PintArray
                new_col = PintArray(df[col], dtype=f"pint[{ureg(df[units_col][0]).u}]")
            else:
                # Make a pd.Series of Quantity in a way that does not throw UnitStrippedWarning
                new_col = pd.Series(data=df[col], name=col) * pd.Series(data=df[units_col].map(lambda x: ureg(x).u), name=col)
            df = df.drop(columns=units_col)
            df[col] = new_col
            units_col = None
    return df


In [12]:
import pandas as pd
import pint
from pint import set_application_registry, Quantity
from pint_pandas import PintArray, PintType
from pint_pandas.pint_array import is_pint_type
from openscm_units import unit_registry
#PintType.ureg = unit_registry
ureg = unit_registry
Q_ = ureg.Quantity
ureg.define("CO2e=CO2=CO2eq=CO2_eq")
ureg.define("USD=[currency]=$")
ureg.define('Millions=1000000')
set_application_registry(ureg)

df_unfccc = requantify_df(df_unfccc)
df_unfccc
# convert to CO2 units to Mt
df_unfccc['ghg_total_without_lulucf'] = df_unfccc['ghg_total_without_lulucf'].pint.to("Mt CO2e")
df_unfccc['gdp'] = df_unfccc['gdp'].pint.to("Millions USD")
df_unfccc
df_unfccc.info(verbose=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149 entries, 0 to 1148
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype                      
---  ------                    --------------  -----                      
 0   country_iso_code          1149 non-null   object                     
 1   validity_date             1149 non-null   int64                      
 2   ghg_total_without_lulucf  1149 non-null   pint[CO2e * megametric_ton]
 3   gdp                       1149 non-null   pint[Millions * USD]       
dtypes: int64(1), object(1), pint[CO2e * megametric_ton](1), pint[Millions * USD](1)
memory usage: 36.0+ KB


In [13]:
# If DF_COL contains Pint quantities (because it is a PintArray or an array of Pint Quantities),
# return a two-column dataframe of magnitudes and units.
# If DF_COL contains no Pint quanities, return it unchanged.

def dequantify_column(df_col: pd.Series):
    if type(df_col.values)==PintArray:
        return pd.DataFrame({df_col.name: df_col.values.quantity.m,
                             df_col.name + "_units": str(df_col.values.dtype.units)},
                            index=df_col.index)
    elif df_col.size==0:
        return df_col
    elif isinstance(df_col.iloc[0], Quantity):
        values = df_col.map(lambda x: (x.m, x.u))
        return pd.DataFrame({df_col.name: df_col.map(lambda x: x.m),
                             df_col.name + "_units": df_col.map(lambda x: str(x.u))},
                            index=df_col.index)
    else:
        return df_col

# Rewrite dataframe DF so that columns containing Pint quantities are represented by a column for the Magnitude and column for the Units.
# The magnitude column retains the original column name and the units column is renamed with a _units suffix.
def dequantify_df(df):
    return pd.concat([dequantify_column(df[col]) for col in df.columns], axis=1)

In [14]:
df_unfccc = dequantify_df(df_unfccc)
df_unfccc
df_unfccc.info(verbose=True)
df_unfccc

#df_result= requantify_df(df_result)
#df_result= dequantify_df(df_result)
#df_result


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149 entries, 0 to 1148
Data columns (total 6 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   country_iso_code                1149 non-null   object 
 1   validity_date                   1149 non-null   int64  
 2   ghg_total_without_lulucf        1149 non-null   float64
 3   ghg_total_without_lulucf_units  1149 non-null   object 
 4   gdp                             1149 non-null   float64
 5   gdp_units                       1149 non-null   object 
dtypes: float64(2), int64(1), object(3)
memory usage: 54.0+ KB


,country_iso_code,validity_date,ghg_total_without_lulucf,ghg_total_without_lulucf_units,gdp,gdp_units
0,JPN,1990,1269.015303,CO2e * megametric_ton,2.466213e+06,Millions * USD
1,JPN,1991,1283.386635,CO2e * megametric_ton,2.636747e+06,Millions * USD
2,JPN,1992,1295.604147,CO2e * megametric_ton,2.719707e+06,Millions * USD
3,JPN,1993,1291.284868,CO2e * megametric_ton,2.769754e+06,Millions * USD
4,JPN,1994,1352.190749,CO2e * megametric_ton,2.856993e+06,Millions * USD
...,...,...,...,...,...,...
1144,CYP,2013,7.918777,CO2e * megametric_ton,2.618148e+04,Millions * USD
1145,CYP,1993,6.798931,CO2e * megametric_ton,9.293831e+03,Millions * USD
1146,CYP,1994,7.049134,CO2e * megametric_ton,1.005234e+04,Millions * USD
1147,CYP,1995,6.985579,CO2e * megametric_ton,1.112138e+04,Millions * USD


<font size=3><b>Merge UNFCCC , Worldbank and OECD data</b> </font>

In [15]:
df_result = pd.merge(df_result,df_unfccc,on=['country_iso_code','validity_date'])  
#df_result
rename_columns = {'ghg_total_without_lulucf':'scope1_value','ghg_total_without_lulucf_units':'scope1_value_units','gdp':'PPP_adjusted_GDP','gdp_units':'PPP_adjusted_GDP_units'}
columns_order = ['country_iso_code','validity_date','scope1_value','scope1_value_units','scope2_value','scope2_value_units','scope3_value','scope3_value_units','PPP_adjusted_GDP','PPP_adjusted_GDP_units']
df_result = df_result.rename(columns=rename_columns).reindex(columns=columns_order)

In [16]:
df_result['scope1_value_units'] = 'Mt CO2e'
#df_result['scope1_value'] = df_result['scope1_value'].round(decimals = 3)
pd.options.display.float_format = '{:.3f}'.format
df_result= df_result.convert_dtypes()
df_result.info(verbose=True)
df_result[df_result['country_iso_code']=='CAN']
#df_result

<class 'pandas.core.frame.DataFrame'>
Int64Index: 934 entries, 0 to 933
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   country_iso_code        934 non-null    string 
 1   validity_date           934 non-null    Int64  
 2   scope1_value            934 non-null    Float64
 3   scope1_value_units      934 non-null    string 
 4   scope2_value            934 non-null    Float64
 5   scope2_value_units      934 non-null    string 
 6   scope3_value            934 non-null    Float64
 7   scope3_value_units      934 non-null    string 
 8   PPP_adjusted_GDP        934 non-null    Float64
 9   PPP_adjusted_GDP_units  934 non-null    string 
dtypes: Float64(4), Int64(1), string(5)
memory usage: 84.8 KB


,country_iso_code,validity_date,scope1_value,scope1_value_units,scope2_value,scope2_value_units,scope3_value,scope3_value_units,PPP_adjusted_GDP,PPP_adjusted_GDP_units
96,CAN,1997,691.195,Mt CO2e,6.362,Mt CO2,202.972,Mt CO2,755390.170,Millions * USD
97,CAN,1998,696.902,Mt CO2e,7.993,Mt CO2,214.108,Mt CO2,793674.328,Millions * USD
98,CAN,1999,709.601,Mt CO2e,9.100,Mt CO2,222.868,Mt CO2,846412.946,Millions * USD
99,CAN,2000,733.511,Mt CO2e,18.191,Mt CO2,236.036,Mt CO2,900996.987,Millions * USD
100,CAN,2001,722.935,Mt CO2e,19.531,Mt CO2,228.887,Mt CO2,937786.776,Millions * USD
101,CAN,2002,726.555,Mt CO2e,8.195,Mt CO2,233.500,Mt CO2,971003.788,Millions * USD
102,CAN,2003,744.832,Mt CO2e,8.220,Mt CO2,232.392,Mt CO2,1023682.523,Millions * USD
103,CAN,2004,745.920,Mt CO2e,8.411,Mt CO2,238.375,Mt CO2,1083609.294,Millions * USD
104,CAN,2005,738.719,Mt CO2e,11.359,Mt CO2,242.704,Mt CO2,1171326.188,Millions * USD
105,CAN,2006,730.102,Mt CO2e,9.075,Mt CO2,237.489,Mt CO2,1241594.040,Millions * USD


Calculate Attribution factor = (Scope1+Scope2+Scope3)/GDP


In [17]:
df_result['attribution_factor']= (df_result['scope1_value'] + df_result['scope2_value'] + df_result['scope3_value']) / df_result['PPP_adjusted_GDP']*100
format_mapper =  {'PPP_adjusted_GDP': '{0:,.0f}',
           'attribution_factor': '{0:.6f}%'}

df_result[df_result['country_iso_code']=='CAN'].style.format(format_mapper)


,country_iso_code,validity_date,scope1_value,scope1_value_units,scope2_value,scope2_value_units,scope3_value,scope3_value_units,PPP_adjusted_GDP,PPP_adjusted_GDP_units,attribution_factor
96,CAN,1997,691.195367,Mt CO2e,6.362000,Mt CO2,202.972000,Mt CO2,"755,390",Millions * USD,0.119214%
97,CAN,1998,696.901519,Mt CO2e,7.993000,Mt CO2,214.108000,Mt CO2,"793,674",Millions * USD,0.115791%
98,CAN,1999,709.600924,Mt CO2e,9.100000,Mt CO2,222.868000,Mt CO2,"846,413",Millions * USD,0.111242%
99,CAN,2000,733.511337,Mt CO2e,18.191000,Mt CO2,236.036000,Mt CO2,"900,997",Millions * USD,0.109627%
100,CAN,2001,722.935012,Mt CO2e,19.531000,Mt CO2,228.887000,Mt CO2,"937,787",Millions * USD,0.103579%
101,CAN,2002,726.555183,Mt CO2e,8.195000,Mt CO2,233.500000,Mt CO2,"971,004",Millions * USD,0.099716%
102,CAN,2003,744.831887,Mt CO2e,8.220000,Mt CO2,232.392000,Mt CO2,"1,023,683",Millions * USD,0.096265%
103,CAN,2004,745.919782,Mt CO2e,8.411000,Mt CO2,238.375000,Mt CO2,"1,083,609",Millions * USD,0.091611%
104,CAN,2005,738.718602,Mt CO2e,11.359000,Mt CO2,242.704000,Mt CO2,"1,171,326",Millions * USD,0.084757%
105,CAN,2006,730.101935,Mt CO2e,9.075000,Mt CO2,237.489000,Mt CO2,"1,241,594",Millions * USD,0.078662%


<font size=3>Save the results in Trino</font>

In [18]:
import osc_ingest_trino as osc
#df_result['validity_date']=pd.to_datetime(df_result['validity_date'], unit='D')
#df_result=df_result.convert_dtypes()
#df_result.info()
columnschema = osc.create_table_schema_pairs(df_result) 
sql = f"""
drop table if exists {ingest_catalog}.{ingest_schema}.{ingest_table}
"""
print(sql)
qres = engine.execute(sql)
print(qres.fetchall())


drop table if exists osc_datacommons_dev.pcaf_sovereign_footprint.sf_total_sovereign_emissions

[(True,)]


In [19]:
df_result.to_sql(ingest_table,
           con=engine,
           schema=ingest_schema,
           if_exists='append',
           index=False,
           method=osc.TrinoBatchInsert(batch_size = 1000, verbose = True))

constructed fully qualified table name as: "pcaf_sovereign_footprint.sf_total_sovereign_emissions"
inserting 934 records
  ('AUS', 1995, 437.27655447052786, 'Mt CO2e', 0.875, 'Mt CO2', 75.42, 'Mt CO2', 378130.489660551, 'Millions * USD', 0.13581860455938444)
  ('AUS', 1996, 443.92826820304475, 'Mt CO2e', 0.974, 'Mt CO2', 85.228, 'Mt CO2', 403005.180162152, 'Millions * USD', 0.13154428138857746)
  ('AUS', 1997, 456.01597642189085, 'Mt CO2e', 0.961, 'Mt CO2', 85.85900000000001, 'Mt CO2', 425381.557676043, 'Millions * USD', 0.1276115446535878)
  ...
  ('USA', 2018, 6671.449400774597, 'Mt CO2e', 12.83, 'Mt CO2', 585.376, 'Mt CO2', 20611860.933999896, 'Millions * USD', 0.03526928220626153)
batch insert result: [(934,)]


In [20]:
df_result.to_csv(ingest_table+".csv")
sql=f"""
select * from {ingest_catalog}.{ingest_schema}.{ingest_table}"""
pd.read_sql(sql, engine)

,country_iso_code,validity_date,scope1_value,scope1_value_units,scope2_value,scope2_value_units,scope3_value,scope3_value_units,ppp_adjusted_gdp,ppp_adjusted_gdp_units,attribution_factor
0,AUS,1995,437.277,Mt CO2e,0.875,Mt CO2,75.420,Mt CO2,378130.490,Millions * USD,0.136
1,AUS,1996,443.928,Mt CO2e,0.974,Mt CO2,85.228,Mt CO2,403005.180,Millions * USD,0.132
2,AUS,1997,456.016,Mt CO2e,0.961,Mt CO2,85.859,Mt CO2,425381.558,Millions * USD,0.128
3,AUS,1998,469.752,Mt CO2e,1.450,Mt CO2,90.130,Mt CO2,453005.215,Millions * USD,0.124
4,AUS,1999,476.392,Mt CO2e,1.760,Mt CO2,93.872,Mt CO2,478757.084,Millions * USD,0.119
...,...,...,...,...,...,...,...,...,...,...,...
929,USA,2014,6824.957,Mt CO2e,13.410,Mt CO2,662.501,Mt CO2,17527163.695,Millions * USD,0.043
930,USA,2015,6671.112,Mt CO2e,12.429,Mt CO2,607.526,Mt CO2,18238300.569,Millions * USD,0.040
931,USA,2016,6520.338,Mt CO2e,12.211,Mt CO2,560.247,Mt CO2,18745075.687,Millions * USD,0.038
932,USA,2017,6483.291,Mt CO2e,12.238,Mt CO2,568.550,Mt CO2,19542979.183,Millions * USD,0.036
